<a href="https://colab.research.google.com/github/hikmatfarhat-ndu/pytorch/blob/main/custom_dataset.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [102]:
import torch
from torch.utils.data import Dataset,DataLoader
import pandas as pd
import numpy as np

In [103]:
!git clone https://github.com/emanhamed/Houses-dataset

fatal: destination path 'Houses-dataset' already exists and is not an empty directory.


In [104]:
df=pd.read_csv("Houses-dataset/Houses Dataset/HousesInfo.txt",header=None,delim_whitespace=True,
               names=["bedrooms","bathrooms","size","zipcode","price"])

In [105]:
def cleanData(df):
    # compute the number of entries per zipcode
    zipcodes=df['zipcode'].value_counts().keys().tolist()
    counts=df['zipcode'].value_counts().tolist()
    #discard all zipcodes ocurring less than 20 times
    for count,zipcode in zip(counts,zipcodes):
      if count<20:
        idx=df[df['zipcode']==zipcode].index
        df.drop(idx,inplace=True)
    return df

In [106]:
dataset=cleanData(df)

In [107]:
dataset.to_csv("cleansedDataset.csv",index=False)

In [108]:
class CustomDataset(Dataset):
  def __init__(self,csvFile,imgDir):
    self.data=pd.read_csv(csvFile)
    self.zipmax=self.data['zipcode'].max()
  def __len__(self):
    return len(self.data)

  def __getitem__(self,idx):
    #
    features=self.data.iloc[idx,[0,1,2]].to_numpy()
    features=torch.tensor(features)
    zip=self.data.iloc[idx,3]
    zip=torch.nn.functional.one_hot(torch.tensor(zip),self.zipmax)
    features=torch.hstack((features,zip))
    return features,self.data.iloc[idx,4]

In [109]:
dataset=CustomDataset("cleansedDataset.csv","images")

In [110]:
itr=iter(dataset)
x=next(itr)
x[0].size()

torch.Size([94534])